In [2]:
import pandas as pd
import numpy as np
import os, sys
import glob

In [3]:
os.remove('./generation_forecast/.DS_Store')


FileNotFoundError: [Errno 2] No such file or directory: './generation_forecast/.DS_Store'

In [4]:
import glob

solaire_files = glob.glob('./generation_forecast/*Solaire*.csv')
eolien_files = glob.glob('./generation_forecast/*Eolien*.csv')
print(solaire_files)

['./generation_forecast/PrevisionSolaire_2020.xls.csv', './generation_forecast/PrevisionSolaire_2021.xls.csv', './generation_forecast/PrevisionSolaire_2017.xls.csv', './generation_forecast/PrevisionSolaire_2023.xls.csv', './generation_forecast/PrevisionSolaire_2019.xls.csv', './generation_forecast/PrevisionSolaire_2018.xls.csv', './generation_forecast/PrevisionSolaire_2022.xls.csv']


In [9]:
files = [solaire_files, eolien_files]
df_list_sol = []
for i, file in enumerate(solaire_files):

    df = pd.read_csv(file, encoding='latin1', sep='\t')
    df_list_sol.append(df)


In [10]:
solaire_files

['./generation_forecast/PrevisionSolaire_2020.xls.csv',
 './generation_forecast/PrevisionSolaire_2021.xls.csv',
 './generation_forecast/PrevisionSolaire_2017.xls.csv',
 './generation_forecast/PrevisionSolaire_2023.xls.csv',
 './generation_forecast/PrevisionSolaire_2019.xls.csv',
 './generation_forecast/PrevisionSolaire_2018.xls.csv',
 './generation_forecast/PrevisionSolaire_2022.xls.csv']

In [11]:
df_list

[                                                   Date  Heure Prévision J-1  \
 0                                            01/01/2020   0:00             0   
 1                                            01/01/2020   1:00             0   
 2                                            01/01/2020   2:00             0   
 3                                            01/01/2020   3:00             0   
 4                                            01/01/2020   4:00             0   
 ...                                                 ...    ...           ...   
 8684                                         31/12/2020  20:00             0   
 8685                                         31/12/2020  21:00             0   
 8686                                         31/12/2020  22:00             0   
 8687                                         31/12/2020  23:00             0   
 8688  RTE ne pourra être tenu responsable de l'usage...    NaN           NaN   
 
      Prévision J  Unnamed

In [7]:

df_solaire = pd.concat(df_list)

#df_solaire = df_solaire.reset_index(inplace=False).drop('index', axis=1)

mask = df_solaire['Heure'].isna()
df_solaire=df_solaire.loc[~mask]

df_solaire['Heure'] = df_solaire['Heure'].apply(lambda x: "0" + x if len(x) == 4 else x)

df_solaire['Date_Heure_France'] = df_solaire['Date'] + ' ' + df_solaire['Heure']

df_solaire['Date_Heure_France'] = pd.to_datetime(df_solaire['Date_Heure_France'], format = "%d/%m/%Y %H:%M")

df_solaire = df_solaire.reset_index(drop=True)
df_solaire = df_solaire.sort_values(by='Date_Heure_France', axis = 0, ascending=True)

df_solaire = df_solaire[['Date_Heure_France', 'Prévision J-1']]
df_solaire.columns =['Date_Heure_France', 'Prévision J-1 Solaire'] 


date_debut = df_solaire['Date_Heure_France'].iloc[0]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_fin = df_solaire['Date_Heure_France'].iloc[-1]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, date_fin , freq='1H')
df_solaire = df_solaire.set_index('Date_Heure_France')
here = date_range.isin(df_solaire.index)
holes= date_range[~here]

for i, idx in enumerate(holes):

    df_solaire.loc[idx, 'Prévision J-1 Solaire'] = 0

df_solaire.loc[df_solaire.index.year == 2023]
date_debut = df_solaire.index[0].tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, periods = len(df_solaire), freq='1H')
df_solaire.index = date_range

df_solaire.index = df_solaire.index.rename('Date UTC')

df_solaire['Prévision J-1 Solaire'].str.startswith('-') == True

df_solaire.loc[df_solaire['Prévision J-1 Solaire'].str.startswith('-') == True] = 0
df_solaire['Prévision J-1 Solaire'] = df_solaire['Prévision J-1 Solaire'].astype(float)

df_solaire.sample(24)

,Prévision J-1 Solaire
Date UTC,
2022-12-23 06:00:00,0.00
2021-01-13 21:00:00,0.00
2019-07-21 21:00:00,0.00
2020-05-17 19:00:00,702.53
2017-10-31 14:00:00,1834.80
2021-03-26 12:00:00,7029.34
2019-06-08 15:00:00,4752.93
2019-06-15 00:00:00,0.00
2017-07-04 10:00:00,0.00


In [7]:

df_eolien = pd.concat(df_list)

#df_eolien = df_eolien.reset_index(inplace=False).drop('index', axis=1)

mask = df_eolien['Heure'].isna()
df_eolien=df_eolien.loc[~mask]

df_eolien['Heure'] = df_eolien['Heure'].apply(lambda x: "0" + x if len(x) == 4 else x)

df_eolien['Date_Heure_France'] = df_eolien['Date'] + ' ' + df_eolien['Heure']

df_eolien['Date_Heure_France'] = pd.to_datetime(df_eolien['Date_Heure_France'], format = "%d/%m/%Y %H:%M")

df_eolien = df_eolien.reset_index(drop=True)
df_eolien = df_eolien.sort_values(by='Date_Heure_France', axis = 0, ascending=True)


df_eolien = df_eolien[['Date_Heure_France', 'Prévision J-1']]
df_eolien.columns = ['Date_Heure_France', 'Prévision J-1 Eolien']

date_debut = df_eolien['Date_Heure_France'].iloc[0]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_fin = df_eolien['Date_Heure_France'].iloc[-1]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, date_fin , freq='1H')
df_eolien = df_eolien.set_index('Date_Heure_France')
here = date_range.isin(df_eolien.index)
holes= date_range[~here]

for i, idx in enumerate(holes):

    df_eolien.loc[idx, 'Prévision J-1 Eolien'] = 0

df_eolien.loc[df_eolien.index.year == 2023]
date_debut = df_eolien.index[0].tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, periods = len(df_eolien), freq='1H')
df_eolien.index = date_range

df_eolien.index = df_eolien.index.rename('Date UTC')

df_eolien['Prévision J-1 Eolien'].str.startswith('-') == True

df_eolien.loc[df_eolien['Prévision J-1 Eolien'].str.startswith('-') == True] = 0
df_eolien['Prévision J-1 Eolien'] = df_eolien['Prévision J-1 Eolien'].astype(float)

df_eolien.sample(24)

,Prévision J-1 Eolien
Date UTC,
2019-03-23 00:00:00,0.00
2022-03-15 10:00:00,7491.46
2021-07-16 21:00:00,75.26
2020-01-18 05:00:00,0.00
2017-03-21 11:00:00,2965.25
2020-03-24 05:00:00,0.00
2019-12-25 02:00:00,0.00
2019-10-28 17:00:00,0.00
2021-01-04 13:00:00,2410.47


In [8]:
df_renew = df_solaire.join(df_eolien)
df_renew.to_csv('/home/infres/dfitton-21/pfr/fil_rouge/examples/datasets/RENEW_FORECAST_FR.csv')

OSError: [Errno 122] Disk quota exceeded

In [10]:
init_data = pd.read_csv("../../../examples/datasets/EPEX_FR_NEW_UTC.csv",
                        index_col='Date')
init_data

,Load forecast,Generation forecast,Price
Date,,,
2017-01-01 00:00:00,73650.0,69539.0,58.23
2017-01-01 01:00:00,72350.0,67376.0,51.95
2017-01-01 02:00:00,68750.0,65412.0,47.27
2017-01-01 03:00:00,65900.0,64557.0,45.49
2017-01-01 04:00:00,65000.0,64552.0,44.50
...,...,...,...
2022-12-31 19:00:00,46950.0,44648.0,7.60
2022-12-31 20:00:00,44350.0,43841.0,3.69
2022-12-31 21:00:00,43950.0,44578.0,1.88


In [11]:
init_data.loc[init_data.isna().any(axis=1)]

,Load forecast,Generation forecast,Price
Date,,,
2019-02-17 23:00:00,59200.0,NaN,39.41
2019-02-18 00:00:00,57100.0,NaN,38.78
2019-02-18 01:00:00,57200.0,NaN,38.78
2019-02-18 02:00:00,54500.0,NaN,38.20
2019-02-18 03:00:00,53700.0,NaN,38.80
...,...,...,...
2021-05-28 17:00:00,46800.0,NaN,83.67
2021-05-28 18:00:00,45150.0,NaN,82.90
2021-05-28 19:00:00,43950.0,NaN,80.10


In [12]:
init_data.index = pd.to_datetime(init_data.index)

In [13]:
all_data = init_data.join(df_renew)

# Pour les valeurs manquantes (Generation), on assigne la valeur Load
all_data['Generation forecast'].loc[all_data.isna().any(axis=1)==True] =all_data['Load forecast'].loc[all_data.isna().any(axis=1)==True]  

In [14]:
price = all_data.pop('Price')
all_data.insert(0,'Price' ,price)


In [15]:
all_data.to_csv('../../../examples/datasets/FR_NEW_UTC_W_RENEW.csv')

In [16]:
all_data[all_data.isna().any(axis=1)]

,Load forecast,Generation forecast,Price,Prévision J-1 Solaire,Prévision J-1 Eolien
Date,,,,,


In [45]:
all_data.loc[:,all_data.isna().any(axis=0)]

""
Date
2017-01-01 00:00:00
2017-01-01 01:00:00
2017-01-01 02:00:00
2017-01-01 03:00:00
2017-01-01 04:00:00
...
2022-12-31 19:00:00
2022-12-31 20:00:00
2022-12-31 21:00:00


In [47]:
all_data.index.isna().sum()

0

In [3]:
import pandas as pd
df = pd.read_csv('../../../examples/experimental_files/fc_nl_datFR_NEW_UTC_W_RENEW_YT2_CW728.csv')

In [12]:
df['Date'].loc[df.isna().any(axis=1)].values[0]

'2021-05-01'